In [1]:
import pydicom
import time
from tqdm import tqdm
from itertools import chain
import os
import numpy
from matplotlib import pyplot, cm
import cv2
import pandas as pd
from keras.utils import to_categorical, Sequence
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import models
from keras import layers
from keras import optimizers

ModuleNotFoundError: No module named 'tqdm'

In [37]:
# Look for all cropped images given in dataframes "paths_dicom_df" in the folder "pth_folder_dicom", 
# apply CLAHE algorithm if wanted, and record new JPEG images in "path_jpg"
# Also creates and record the data frame necessary for the Keras image generator
def dicom_to_jpg_with_df(paths_dicom_df, pth_folder_dicom, path_jpg, labels_df_name, ENHANCE=True):
    clahe1 = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(6,6)) #CLAHE for local contrast enhancing
    clahe2 = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clahe3 = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(10,10))
    threshold1 = 250*250
    threshold2 = 400*400
    i=0
    dict_for_df = {'filename':[], 'pathology':[]}
    for df_path in paths_dicom_df:
        df = pd.read_csv(df_path, sep=';', header=0)
        df.dropna(0, inplace=True, how='all') # Remove fully empty rows
        df.reset_index(drop=True, inplace=True)
        for index, row in df.iterrows():
            foldername = row["cropped image file path"]
            arg_list = foldername.split('/')
            path = os.path.join(path_dicom_folder_test, arg_list[0])
            dict_for_df['filename'].append(arg_list[0])
            dict_for_df['pathology'].append(row.pathology)
            for dirName, subdirList, fileList in os.walk(path):
                for filename in fileList:
                    filepath = os.path.join(dirName, filename)
                    dic = pydicom.read_file(filepath)
                    if (dic.SeriesDescription == "cropped images"):  # check whether the file's segmented image
                        pixels = dic.pixel_array
                        size = pixels.size
                        if ENHANCE:
                            if size<threshold1:
                                pixels = clahe1.apply(pixels) 
                            elif size<threshold2:
                                pixels = clahe2.apply(pixels)
                            else:
                                pixels = clahe3.apply(pixels)
                        cv2.imwrite(os.path.join(path_jpg, arg_list[0]), pixels/256., [int(cv2.IMWRITE_JPEG_QUALITY), 100])
                        i+=1
    df_returned = pd.DataFrame(data=dict_for_df)
    df.to_pickle(labels_df_name)

In [38]:
path_train_mass_df = ""
path_train_calc_df = ""
path_test_mass_df = "./../CBIS-DDSM/mass_case_description_test_set.csv"
path_test_calc_df = ""

path_dicom_folder_test = "./../CBIS-DDSM/Mix_crop_test_mass/CBIS-DDSM"

path_jpg_folder_test = "./../CBIS-DDSM/crop_test_mass_jpg/"

dicom_to_jpg([path_test_mass_df], path_dicom_folder_test, path_jpg_folder_test)